In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [2]:
# load raw data
data = pd.read_stata('data/dataset.dta')
data.vote_trump = data.vote_trump.cat.add_categories(['Unknow']).fillna('Unknow')

In [3]:
# trump voter income levels
data_income = data[['id', 'vote_trump', 'income', 'worry_finances']]

In [4]:
# drop income == 1 rows
data_income = data_income.loc[data_income.income != 0].dropna()

In [5]:
data_income.groupby('vote_trump').mean()

,id,income
vote_trump,,
No,374877.87500,87196.234375
Yes,376291.78125,82576.718750
Unknow,376015.43750,66147.007812


In [6]:
data_income.groupby('worry_finances').mean()

,id,income
worry_finances,,
Not at all worried,363782.31250,104896.773438
A little worried,374507.46875,91736.101562
Moderately worried,379372.56250,72586.882812
Very worried,383095.31250,54695.074219
Extremely worried,380203.31250,55817.843750


In [7]:
data_income.groupby('worry_finances').count()

,id,vote_trump,income
worry_finances,,,
Not at all worried,565,565,565
A little worried,927,927,927
Moderately worried,941,941,941
Very worried,371,371,371
Extremely worried,318,318,318


In [8]:
data_income.describe()

,id,income
count,3122.000000,3122.000000
mean,375633.437500,80285.820312
std,45847.035156,62601.585938
min,300001.000000,10002.000000
25%,302583.500000,35114.500000
50%,402440.000000,63166.500000
75%,405030.000000,103721.500000
max,407791.000000,299378.000000


In [13]:
data_age = data[['id', 'vote_trump', 'age']].dropna()

In [26]:
# replace 90. Age 90 or older to 90
data_age.age = data_age.age.replace('90. Age 90 or older', 90)

In [27]:
data_age.groupby('vote_trump').mean()

,id,age
vote_trump,,
No,374757.53125,49.178546
Yes,375948.93750,53.991023
Unknow,375066.25000,44.106291


In [29]:
data_age.describe()

,id,age
count,3453.000000,3453.000000
mean,375224.468750,49.376774
std,46066.738281,17.580693
min,300001.000000,18.000000
25%,302542.000000,34.000000
50%,402437.000000,50.000000
75%,405080.000000,63.000000
max,407799.000000,90.000000
